In [16]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [9]:
print(torchvision.__version__)

NameError: name 'torchvision' is not defined

In [5]:
%pip uninstall torchvision

Found existing installation: torchvision 0.20.1
Uninstalling torchvision-0.20.1:
  Would remove:
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision-0.20.1.dist-info/*
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision.libs/libcudart.41118559.so.12
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision.libs/libnvjpeg.02b6d700.so.12
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision.libs/libwebp.4a54d2c8.so.4
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision.libs/libz.5f199d92.so.1
    /usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [1]:


class CharacterDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.labels = self.data.iloc[:, 0].values
        self.images = self.data.iloc[:, 1:].values.reshape(-1, 28, 28).astype('float32')
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the dataset
csv_file = 'characters.csv'
dataset = CharacterDataset(csv_file, transform=transform)

# Create DataLoader
batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example of iterating through the DataLoader
for images, labels in dataloader:
    print(images.shape, labels.shape)
    break

ModuleNotFoundError: No module named 'torchvision'